<a href="https://colab.research.google.com/github/ihab09/CE888/blob/main/lab4/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
# There are 24,983 rows each represent a user
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv", header=None)
original_df = df.copy()
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

In [ ]:
# Ratings are real values ranging from -10.00 to +10.00. Replace the value "99" with NaN because it corresponds to "not rated".
df = df.replace(99.00, np.NaN)
# df = df.applymap(lambda x: np.nan if x == 99.00 else x)
df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74.0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,100.0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49.0,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,48.0,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,91.0,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100.0,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91.0,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39.0,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,37.0,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Drop the first column because it represents the number of jokes rated by that user while keeping the next 100 columns because they're the ratings for jokes 01 - 100.
df.drop(0, inplace=True, axis=1)
df

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.describe() 

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
count,16452.000000,18184.000000,16642.000000,15608.000000,24977.000000,20431.000000,24976.000000,24981.000000,15752.000000,20324.000000,...,9672.000000,9889.000000,9988.000000,10200.000000,10265.000000,10370.000000,10520.000000,10666.000000,10910.000000,9943.000000
mean,0.904573,0.208460,0.316574,-1.449114,0.403556,1.594447,-0.429662,-0.624085,-0.557031,1.301231,...,2.038527,1.179744,2.504041,1.173482,1.039130,1.528003,1.673750,0.767692,-0.031244,1.354711
std,5.171798,5.597327,5.399806,5.234654,5.255700,4.900540,5.495931,4.929144,5.162224,5.087359,...,5.060445,5.127291,4.611339,5.140307,5.167043,4.852069,5.025076,5.393833,5.176806,5.300409
min,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,-2.670000,-4.420000,-4.030000,-5.870000,-3.690000,-1.310000,-5.290000,-4.610000,-4.760000,-2.090000,...,-0.932500,-2.380000,-0.100000,-2.280000,-2.570000,-1.347500,-1.170000,-3.400000,-4.170000,-2.330000
50%,1.600000,0.830000,0.780000,-1.460000,1.120000,2.230000,-0.050000,-0.290000,-0.190000,1.940000,...,2.860000,1.800000,3.160000,1.750000,1.650000,2.140000,2.480000,1.460000,0.290000,2.140000
75%,4.900000,4.820000,4.660000,2.520000,4.470000,5.440000,3.880000,3.060000,3.300000,5.390000,...,6.120000,5.290000,6.070000,5.290000,5.100000,5.277500,5.630000,5.150000,3.980000,5.730000
max,9.370000,9.370000,9.370000,9.370000,9.370000,9.470000,9.610000,9.760000,9.900000,9.370000,...,9.560000,9.420000,9.370000,9.420000,9.370000,9.370000,9.370000,9.370000,9.370000,9.370000


In [ ]:
#  Use pandas to find the best- and the worst-rated jokes
joke_avg_rating = df.mean(axis=0)
joke_avg_rating

1      0.904573
2      0.208460
3      0.316574
4     -1.449114
5      0.403556
         ...   
96     1.528003
97     1.673750
98     0.767692
99    -0.031244
100    1.354711
Length: 100, dtype: float64

In [ ]:
joke_avg_rating[1]

0.9045733041575473

In [ ]:
joke_avg_rating[[1]]

1    0.904573
dtype: float64

In [ ]:
joke_avg_rating.shape

(100,)

In [ ]:
joke_avg_rating.max()

3.6650848950824937

In [ ]:
joke_avg_rating.min()

-3.8338796373689923

In [ ]:
joke_avg_rating[joke_avg_rating==3.6650848950824937].index.item()

50

In [ ]:
max_index = joke_avg_rating.idxmax()
print("The best rated joke id:", max_index)

The best rated joke id: 50


In [ ]:
min_index = joke_avg_rating.idxmin()
min_index
print("The worst rated joke id:", min_index)

The worst rated joke id: 58


In [ ]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  # rated = np.where(arr!=99)
  rated = np.where(~np.isnan(orig))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [ ]:
arr = df.values
print(arr.shape)
arr

(24983, 100)


array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [ ]:
new_arr, idx = replace(arr, 0.1)
print(new_arr.shape)
new_arr

(24983, 100)


array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, 99.  ,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [ ]:
idx

(array([20003,  5502, 19524, ...,  1150, 11969, 20612], dtype=int32),
 array([21, 91,  5, ..., 16,  6, 62], dtype=int32))

In [ ]:
arr[idx[0][0], idx[1][0]]

6.84

In [ ]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [ ]:
n_latent_factors = 2

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [ ]:
print(latent_user_preferences.shape)
latent_user_preferences

(24983, 2)


array([[0.25157561, 0.82000357],
       [0.88647615, 0.22408423],
       [0.98463546, 0.11004589],
       ...,
       [0.2780965 , 0.66185229],
       [0.47121408, 0.46115007],
       [0.42343969, 0.43520261]])

In [ ]:
print(latent_item_features.shape)
latent_item_features

(100, 2)


array([[0.38709627, 0.10700243],
       [0.62974255, 0.56691986],
       [0.49253916, 0.89841243],
       [0.03535205, 0.82379004],
       [0.84877714, 0.16816071],
       [0.57254957, 0.27084235],
       [0.8305643 , 0.46094393],
       [0.60678145, 0.76350916],
       [0.91620154, 0.2301513 ],
       [0.61672565, 0.08005412],
       [0.24979783, 0.31872485],
       [0.16387615, 0.45327266],
       [0.82460282, 0.50720732],
       [0.93721175, 0.96514076],
       [0.22323824, 0.31492261],
       [0.43244486, 0.06820658],
       [0.73463415, 0.73560391],
       [0.31254061, 0.85782739],
       [0.95120503, 0.77511621],
       [0.45126588, 0.18089247],
       [0.1641699 , 0.75541095],
       [0.50223931, 0.49505088],
       [0.93429872, 0.54620335],
       [0.59673018, 0.74380199],
       [0.6357165 , 0.9051806 ],
       [0.75238228, 0.89239543],
       [0.12175318, 0.72158414],
       [0.63631681, 0.48764088],
       [0.78180427, 0.10010368],
       [0.33325238, 0.83968492],
       [0.

In [ ]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = new_arr[user_id, item_id]
                if rating != 99 and not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [ ]:
num_iter = 100
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 10000), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

Iteration 0/100:	MSE=21.330565


In [ ]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 0.60108639,  0.46563115,  0.16905104, ...,  0.62336175,
        -0.06359674,  0.8543841 ],
       [ 1.53662555,  1.30684054,  1.49101379, ...,  1.85487891,
         1.44402058,  2.20593154],
       [ 3.66318816,  3.07968444,  3.22982836, ...,  4.34176518,
         2.94986938,  5.25208327],
       ...,
       [ 0.57323351,  0.45766486,  0.28492167, ...,  0.62500502,
         0.12704725,  0.81733765],
       [ 1.21856355,  1.00916523,  0.93538424, ...,  1.40998444,
         0.7703386 ,  1.74425266],
       [ 2.30467467,  1.91390945,  1.81700361, ...,  2.67853265,
         1.52963484,  3.2998979 ]])

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns
comparison_data

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.6010863877258216)","(8.79, 0.4656311533909185)","(-9.66, 0.16905103995286191)","(-8.16, -0.8352866034160444)","(-7.52, 0.018903082229645037)","(-8.5, 1.097672710940585)","(-9.85, -0.30091184929935016)","(4.17, -0.4581779013786699)","(-8.98, -0.37382925049785837)","(-4.76, 0.8616060182179166)",...,"(2.82, 1.1841173809766181)","(nan, 0.47404693743778664)","(nan, 1.716441477104562)","(nan, 0.7542298122108656)","(nan, 0.5183540258567828)","(nan, 1.114946280263378)","(-5.63, 0.8122200763871598)","(nan, 0.6233617526481886)","(nan, -0.06359674368951897)","(nan, 0.8543841044573546)"
1,"(4.08, 1.5366255466490577)","(-0.29, 1.3068405351844725)","(6.36, 1.4910137933617056)","(4.37, 0.70645913407371)","(-2.38, 1.4590557482613489)","(-9.66, 2.370936263437521)","(-0.73, 0.7841636802081294)","(-5.34, 0.2782064384269563)","(8.88, 1.053903075017722)","(9.22, 1.6775006276145086)",...,"(2.82, 2.2319773037640447)","(-4.95, 2.2367983895119266)","(-0.29, 2.3447358674786813)","(7.86, 1.701870322479145)","(-0.19, 1.8541778459194944)","(-2.14, 1.8867389702410275)","(3.06, 2.1841661472042846)","(0.34, 1.8548789119762508)","(-4.32, 1.4440205807926794)","(1.07, 2.2059315357255964)"
2,"(nan, 3.663188158714516)","(nan, 3.0796844412540474)","(nan, 3.2298283598942223)","(nan, 0.8128935425604732)","(9.03, 3.0457619251094865)","(9.27, 5.785531393876772)","(9.03, 1.3931296035308527)","(9.27, 0.21882954312165073)","(nan, 1.896319284231058)","(nan, 4.160015804027448)",...,"(nan, 5.5646184350508285)","(nan, 5.018113702831305)","(nan, 6.216066200729956)","(9.08, 4.126483399255)","(nan, 4.258008723310508)","(nan, 4.793229948469311)","(nan, 5.173823103515142)","(nan, 4.341765177815998)","(nan, 2.9498693814206547)","(nan, 5.252083269555643)"
3,"(nan, 1.673770826734225)","(8.35, 1.3774288323101118)","(nan, 1.2055420177706362)","(nan, -0.3538055987025377)","(1.8, 1.0316382743822234)","(8.16, 2.754560049425266)","(-2.82, 0.24010947605397448)","(6.21, -0.2699269663498014)","(nan, 0.3536159977445654)","(1.84, 2.0347910052903564)",...,"(nan, 2.74548026491762)","(nan, 2.0312931183564507)","(nan, 3.361649461847907)","(0.53, 1.943198109278126)","(nan, 1.8105398165452125)","(nan, 2.435998212178057)","(nan, 2.3364951974830697)","(nan, 1.9171376227871204)","(nan, 0.937837519093958)","(nan, 2.3942068032642707)"
4,"(8.5, 2.234145628973159)","(4.61, 1.8495513894194986)","(-4.17, 1.7087949393740318)","(-5.39, -0.2048394428154416)","(1.36, 1.5097815135551753)","(1.6, 3.6358302962136557)","(7.04, 0.4666777983390143)","(4.61, -0.22389684374869212)","(-0.44, 0.6611104567160304)","(5.73, 2.6666195187543984)",...,"(5.19, 3.589838771955658)","(5.58, 2.807814584280852)","(4.27, 4.294852743171886)","(5.19, 2.5724856429626475)","(5.73, 2.466489408966595)","(1.55, 3.1608952013338687)","(3.11, 3.128892465695164)","(6.55, 2.583580792236042)","(1.8, 1.4030081381605812)","(1.6, 3.197830803908578)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,"(0.44, 2.8824980020860957)","(7.43, 2.4251766007579927)","(9.08, 2.5581041651840484)","(2.33, 0.6842285713814369)","(3.2, 2.4187852388136712)","(6.75, 4.545706217386937)","(-8.79, 1.1205963023879948)","(-0.53, 0.19493023238870083)","(-8.74, 1.523702572763794)","(7.23, 3.26520573926145)",...,"(8.83, 4.366227508526446)","(-1.21, 3.964742350261296)","(9.22, 4.85926256778969)","(-6.7, 3.243507782709)","(8.45, 3.358850582213758)","(9.03, 3.756593857319865)","(6.55, 4.072880836818963)","(8.69, 3.420557197719562)","(8.79, 2.346401525151001)","(7.43, 4.133112998653555)"
24979,"(9.13, 0.5071552119958446)","(-8.16, 0.43342780607893927)","(8.59, 0.5112971938986208)","(9.08, 0.2846821388622697)","(0.87, 0.5071288893995488)","(-8.93, 0.7746258981302337)","(-3.5, 0.28697107772325725)","(5.78, 0.11809857031983755)","(-8.11, 0.38426664023849)","(4.9, 0.5441303668739645)",...,"(-1.17, 0.7222377292844168)","(-5.73, 0.7568247174585568)","(-1.46, 0.73682659633613

# Quiz

In [ ]:
# In the jokes dataset, what's the rating that user 11740 has given to joke 29?
original_df.iloc[11741 , 30]

-6.8

In [ ]:
mov_items = pd.read_excel('movies_latent_factors.xlsx', sheet_name='Items')
mov_items

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [ ]:
mov_users = pd.read_excel('movies_latent_factors.xlsx', sheet_name='Users')
mov_users

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669
5,783,-0.020301,-0.031919,-0.036955,0.033690,0.000174,-0.003178,0.054474,0.045424,-0.053308,0.003437,-0.041750,0.059725,-0.021647,0.039873,-0.061857
6,3878,-0.091462,0.215879,-0.180453,0.085408,-0.321094,0.227947,0.053767,0.038017,0.097141,-0.139872,0.029253,0.214467,0.189250,0.065990,0.102776
7,768,0.000819,-0.009229,-0.019228,0.002703,0.012869,0.006655,0.007687,0.018747,0.014705,-0.009256,-0.010116,-0.010051,-0.002108,0.043223,-0.006670
8,4469,-0.030528,-0.011537,-0.042822,-0.014378,0.031338,0.012297,-0.000038,-0.010264,-0.006781,-0.019950,-0.014435,0.010926,0.024400,0.005318,-0.007983
9,1882,-0.083093,-0.029160,0.013748,0.022716,-0.062732,-0.080670,-0.019723,0.013253,-0.091305,-0.090687,0.030638,-0.113364,-0.021325,0.181508,0.022357


In [ ]:
# In the movies dataset, what's the loading (i.e., value) of Factor4 for the movie with ID=857?
mov_items.loc[mov_items['Movie ID'] == 857, 'Factor4']

69   -0.088227
Name: Factor4, dtype: float64

In [ ]:
# In the movies dataset, which of the following titles corresponds to the movie with the lowest loading of factor 1?
mov_items.loc[mov_items['Factor1'] == mov_items['Factor1'].min() , 'Title']

50    The Matrix (1999)
Name: Title, dtype: object

In the movies dataset, predict the rating that the user with ID 525 will give to the movie with ID 585. Give your answer with two decimal values (e.g., 0.12).

In [ ]:
mov_factor_vals = mov_items.loc[mov_items['Movie ID'] == 585]
mov_factor_vals

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
46,585,Monsters Inc. (2001),-0.342669,-0.140318,0.218295,1.950883,-1.265953,-0.503642,0.253365,0.071408,-0.962871,1.156963,0.258682,-0.171433,-0.569189,-0.024884,0.854865


In [ ]:
mov_factor_vals.drop('Title', inplace=True, axis=1)
mov_factor_vals.drop('Movie ID', inplace=True, axis=1)
mov_factor_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
46,-0.342669,-0.140318,0.218295,1.950883,-1.265953,-0.503642,0.253365,0.071408,-0.962871,1.156963,0.258682,-0.171433,-0.569189,-0.024884,0.854865


In [ ]:
user_factor_vals = mov_users.loc[mov_users['User'] == 525]
user_factor_vals

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
20,525,-0.04029,0.014383,-0.098967,-0.018957,0.012388,0.035546,0.000903,0.012835,-0.02108,0.018742,-0.050483,-0.025394,0.034079,0.03833,-0.001341


In [ ]:
user_factor_vals.drop('User', inplace=True, axis=1)
user_factor_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
20,-0.04029,0.014383,-0.098967,-0.018957,0.012388,0.035546,0.000903,0.012835,-0.02108,0.018742,-0.050483,-0.025394,0.034079,0.03833,-0.001341


In [ ]:
predictions = user_factor_vals.dot(mov_factor_vals.T)
predictions

,46
20,-0.067459


In the movies dataset, predict the rating that the user with ID 3806 will give to the movie with ID 24. Give your answer with two decimal values (e.g., 0.12).

In [ ]:
mov_factor_vals = mov_items.loc[mov_items['Movie ID'] == 24]
mov_factor_vals

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
5,24,Kill Bill: Vol. 1 (2003),-0.645019,1.99744,2.085625,-1.686559,-0.047874,-1.046274,1.613575,4.630243,0.276288,0.408997,-1.148072,0.269853,0.724543,-0.385681,1.036656


In [ ]:
mov_factor_vals.drop('Title', inplace=True, axis=1)
mov_factor_vals.drop('Movie ID', inplace=True, axis=1)
mov_factor_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
5,-0.645019,1.99744,2.085625,-1.686559,-0.047874,-1.046274,1.613575,4.630243,0.276288,0.408997,-1.148072,0.269853,0.724543,-0.385681,1.036656


In [ ]:
user_factor_vals = mov_users.loc[mov_users['User'] == 3806]
user_factor_vals

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
12,3806,-0.042344,-0.040755,-0.05572,-0.073726,0.018203,0.09194,0.123959,-0.157403,-0.020388,0.082493,0.223057,-0.040915,-0.130308,-0.021816,0.034549


In [ ]:
user_factor_vals.drop('User', inplace=True, axis=1)
user_factor_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
12,-0.042344,-0.040755,-0.05572,-0.073726,0.018203,0.09194,0.123959,-0.157403,-0.020388,0.082493,0.223057,-0.040915,-0.130308,-0.021816,0.034549


In [ ]:
predictions = user_factor_vals.dot(mov_factor_vals.T)
predictions

,5
12,-0.961029


# In the movies dataset, if you had to recommend 3 movies to the user with ID=5323, which ones would you choose?

In [ ]:
user_factor_vals = mov_users.loc[mov_users['User'] == 5323]
user_factor_vals

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.01946,0.108372,0.054471,-0.109552


In [ ]:
user_factor_vals.drop('User', inplace=True, axis=1)
user_factor_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
2,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.01946,0.108372,0.054471,-0.109552


In [ ]:
mov_items_arr = mov_items.copy()
mov_items_arr

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [ ]:
mov_items_arr.drop('Title', inplace=True, axis=1)
mov_items_arr.drop('Movie ID', inplace=True, axis=1)
mov_items_arr

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [ ]:
predictions = mov_items_arr.dot(user_factor_vals.T)
predictions

,2
0,0.214068
1,0.154537
2,0.261073
3,-0.187174
4,0.502264
...,...
95,0.046301
96,0.242631
97,-0.083633
98,-0.163929


In [ ]:
largest = predictions.stack().nlargest(3)
largest

21  2    0.759871
27  2    0.694714
91  2    0.667856
dtype: float64

In [ ]:
mov_items

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [ ]:
mov_items.loc[21]

Movie ID                       155
Title       The Dark Knight (2008)
Factor1                   -2.16936
Factor2                   -1.13158
Factor3                   -0.23918
Factor4                  -0.556938
Factor5                   -1.47359
Factor6                  -0.739988
Factor7                    1.35149
Factor8                   -1.67913
Factor9                   0.133108
Factor10                 -0.943399
Factor11                  -1.13559
Factor12                 -0.179583
Factor13                   2.19346
Factor14                  0.720142
Factor15                  0.445634
Name: 21, dtype: object

In [ ]:
mov_items.loc[27]

Movie ID                     238
Title       The Godfather (1972)
Factor1                 -2.61146
Factor2               0.00555141
Factor3                 -1.05533
Factor4                  0.15095
Factor5                 0.233674
Factor6                 0.266476
Factor7                 0.605631
Factor8                 0.595702
Factor9                -0.112474
Factor10                  -1.489
Factor11               -0.378031
Factor12                 0.30331
Factor13               -0.591693
Factor14                0.400705
Factor15                -2.26783
Name: 27, dtype: object

In [ ]:
mov_items.loc[91]

Movie ID                    8587
Title       The Lion King (1994)
Factor1                -0.195123
Factor2                -0.920866
Factor3                 -1.08696
Factor4                  2.53222
Factor5                -0.239391
Factor6                0.0343659
Factor7                 0.652424
Factor8                 0.649638
Factor9                -0.385725
Factor10                0.219746
Factor11               -0.804286
Factor12               -0.662137
Factor13                 1.35416
Factor14                 0.86747
Factor15               -0.438389
Name: 91, dtype: object

# In the movies dataset, if you had to recommend 2 movies to the user with ID=525, which ones would you choose?

In [ ]:
user_factor_vals = mov_users.loc[mov_users['User'] == 525]
user_factor_vals

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
20,525,-0.04029,0.014383,-0.098967,-0.018957,0.012388,0.035546,0.000903,0.012835,-0.02108,0.018742,-0.050483,-0.025394,0.034079,0.03833,-0.001341


In [ ]:
user_factor_vals.drop('User', inplace=True, axis=1)
user_factor_vals

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
20,-0.04029,0.014383,-0.098967,-0.018957,0.012388,0.035546,0.000903,0.012835,-0.02108,0.018742,-0.050483,-0.025394,0.034079,0.03833,-0.001341


In [ ]:
mov_items_arr = mov_items.copy()
mov_items_arr

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [ ]:
mov_items_arr.drop('Title', inplace=True, axis=1)
mov_items_arr.drop('Movie ID', inplace=True, axis=1)
mov_items_arr

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130


In [ ]:
predictions = mov_items_arr.dot(user_factor_vals.T)
predictions

,20
0,0.002598
1,0.022087
2,0.234905
3,0.054462
4,-0.038369
...,...
95,-0.091585
96,0.039609
97,-0.172472
98,-0.190195


In [ ]:
largest = predictions.stack().nlargest(2)
largest

10  20    0.307823
47  20    0.280349
dtype: float64

In [ ]:
mov_items.loc[10]

Movie ID                  98
Title       Gladiator (2000)
Factor1             -1.36485
Factor2             -1.30414
Factor3             -1.94162
Factor4            -0.661007
Factor5             0.128877
Factor6           0.00704816
Factor7             0.371097
Factor8            0.0860265
Factor9            -0.133839
Factor10            0.548053
Factor11           -0.827216
Factor12            0.141476
Factor13            0.735983
Factor14             -0.3381
Factor15           -0.175565
Name: 10, dtype: object

In [ ]:
mov_items.loc[47]

Movie ID               597
Title       Titanic (1997)
Factor1            1.56787
Factor2           -1.51695
Factor3           -2.52338
Factor4            1.20448
Factor5            1.72954
Factor6         -0.0889414
Factor7           0.102822
Factor8            1.74779
Factor9            1.81696
Factor10          -2.98681
Factor11         -0.589441
Factor12          -2.56858
Factor13          0.370387
Factor14           2.21593
Factor15          0.456127
Name: 47, dtype: object